<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/chargpt_coin_tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import time

# CoinGecko API erişim bilgileri
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0'
}

def get_coingecko_data(symbol, vs_currency='usd'):
    """CoinGecko API üzerinden kripto para verisini çeker."""
    params = {
        'vs_currency': vs_currency,
        'ids': symbol.lower(),
        'order': 'market_cap_desc',
        'per_page': 1,
        'page': 1,
        'sparkline': False
    }

    try:
        response = requests.get(COINGECKO_URL, params=params, headers=HEADERS, timeout=15)
        response.raise_for_status()
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            df = df[['id', 'current_price', 'high_24h', 'low_24h']]
            df.rename(columns={'current_price': 'close', 'high_24h': 'high', 'low_24h': 'low'}, inplace=True)
            df['timestamp'] = pd.to_datetime(time.time(), unit='s')
            return df
    except requests.exceptions.RequestException as e:
        print(f"API Hatası ({symbol}): {e}")

    return pd.DataFrame()

def calculate_devisso_trend(df, ma_period=20):
    """Basit hareketli ortalama (SMA) hesaplayarak trend sinyali üretir."""
    if df.empty:
        return df

    df['ma'] = df['close'].rolling(ma_period, min_periods=1).mean()
    df['signal'] = np.where(df['close'] > df['ma'], 'BUY', 'SELL')
    return df

def scan_coins(coin_list):
    """Belirtilen coin listesindeki trend sinyallerini tarar."""
    signals = []
    for coin in coin_list:
        time.sleep(2)  # Rate limit koruması
        data = get_coingecko_data(coin)
        if not data.empty:
            result = calculate_devisso_trend(data)
            if not result.empty:
                last_signal = result.iloc[-1]['signal']
                signals.append((coin.upper(), last_signal))
    return signals

# Çalıştırma
coin_list = ['bitcoin', 'ethereum', 'binancecoin', 'cardano', 'solana']
while True:
    print(f"\n--- Tarama Zamanı: {time.ctime()} ---")
    signals = scan_coins(coin_list)
    if signals:
        for coin, action in signals:
            print(f"{coin} - {action}")
    else:
        print("Sinyal yok")
    time.sleep(300)  # 5 dakikada bir tarama



--- Tarama Zamanı: Mon Feb  3 18:09:21 2025 ---
BITCOIN - SELL
ETHEREUM - SELL
BINANCECOIN - SELL
CARDANO - SELL
SOLANA - SELL
